In [1]:
# Smart Factory Energy Prediction Challenge
# Jupyter Notebook for EDA, Modeling, and Recommendations

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# 1. Load and Clean Data
def load_data(file_path='data/data.csv'):
    """Load the dataset and return a DataFrame."""
    df = pd.read_csv(file_path)
    print(f"Dataset Shape: {df.shape}")
    return df

def clean_data(df):
    """Convert object columns to numeric and handle missing values."""
    # Identify object columns (excluding timestamp)
    object_cols = df.select_dtypes(include=['object']).columns
    object_cols = [col for col in object_cols if col != 'timestamp']
    
    # Convert object columns to numeric, coercing errors to NaN
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Fill missing values with median for numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    
    print("\nData Types After Cleaning:")
    print(df.dtypes)
    print("\nMissing Values After Cleaning:")
    print(df.isnull().sum())
    
    return df

# Load data
df = load_data()

# Clean data
df = clean_data(df)

# 2. Data Preprocessing
def preprocess_data(df):
    """Preprocess the data: encode timestamps and sort."""
    # Convert timestamp to datetime and sort
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')

    # Extract time-based features
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

    # Drop original timestamp
    df = df.drop('timestamp', axis=1)

    return df

# Preprocess data
df_processed = preprocess_data(df)

# 3. Exploratory Data Analysis (EDA)
def perform_eda(df):
    """Perform exploratory data analysis and visualize key patterns."""
    # Summary statistics
    print("\nSummary Statistics:")
    display(df.describe())

    # Correlation matrix (only numeric columns)
    plt.figure(figsize=(12, 8))
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    sns.heatmap(numeric_df.corr(), annot=False, cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.savefig('correlation_matrix.png')
    plt.close()

    # Distribution of target variable
    plt.figure(figsize=(8, 6))
    sns.histplot(df['equipment_energy_consumption'], kde=True)
    plt.title('Distribution of Equipment Energy Consumption')
    plt.savefig('target_distribution.png')
    plt.close()

    # Scatter plots for random variables
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(df['random_variable1'], df['equipment_energy_consumption'], alpha=0.5)
    plt.title('Random Variable 1 vs Energy Consumption')
    plt.subplot(1, 2, 2)
    plt.scatter(df['random_variable2'], df['equipment_energy_consumption'], alpha=0.5)
    plt.title('Random Variable 2 vs Energy Consumption')
    plt.tight_layout()
    plt.savefig('random_variables.png')
    plt.close()

# Perform EDA
perform_eda(df_processed)

# 4. Feature Selection
def select_features(X, y):
    """Perform feature selection using RandomForest importance."""
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    # Feature importance plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x=rf.feature_importances_, y=X.columns)
    plt.title('Feature Importance')
    plt.savefig('feature_importance.png')
    plt.close()

    # Select features above mean importance threshold
    selector = SelectFromModel(rf, threshold='mean')
    selector.fit(X, y)
    selected_features = X.columns[selector.get_support()].tolist()
    
    print("\nSelected Features:")
    print(selected_features)
    
    return selected_features

# Split features and target
X = df_processed.drop('equipment_energy_consumption', axis=1)
y = df_processed['equipment_energy_consumption']

# Select features
selected_features = select_features(X, y)
X_selected = X[selected_features]

# 5. Model Development
def train_and_evaluate_model(X, y, model_name):
    """Train and evaluate a regression model with cross-validation."""
    # Chronological split for time series data
    train_size = int(0.8 * len(X))
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train model
    if model_name == 'RandomForest':
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_name == 'XGBoost':
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    else:
        raise ValueError("Unsupported model name")

    model.fit(X_train_scaled, y_train)

    # Evaluate model
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"\nModel Performance ({model_name}):")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")

    # Cross-validation for time series
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []
    for train_idx, val_idx in tscv.split(X):
        X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
        X_train_cv_scaled = scaler.fit_transform(X_train_cv)
        X_val_cv_scaled = scaler.transform(X_val_cv)
        model.fit(X_train_cv_scaled, y_train_cv)
        y_pred_cv = model.predict(X_val_cv_scaled)
        cv_scores.append(r2_score(y_val_cv, y_pred_cv))

    print("\nCross-Validation R² Scores:")
    print(f"Mean: {np.mean(cv_scores):.4f}, Std: {np.std(cv_scores):.4f}")

    return model, scaler, X_train.columns

# Train and evaluate models
print("\nTraining Random Forest...")
rf_model, rf_scaler, rf_features = train_and_evaluate_model(X_selected, y, 'RandomForest')

print("\nTraining XGBoost...")
xgb_model, xgb_scaler, xgb_features = train_and_evaluate_model(X_selected, y, 'XGBoost')

# 6. Insights and Recommendations
def generate_insights(df, model, feature_names):
    """Generate actionable insights and recommendations."""
    insights = """
# Smart Factory Energy Prediction: Insights and Recommendations

## Approach
We conducted exploratory data analysis to understand sensor data patterns, preprocessed the data by handling missing values and extracting time-based features, and split the data chronologically to respect its time series nature. Feature selection was performed using Random Forest importance, and two models (Random Forest and XGBoost) were trained and evaluated using RMSE, MAE, and R² metrics.

## Key Findings
- **Feature Importance**: Environmental factors such as temperature and humidity in specific zones (e.g., zone1_temperature, zone1_humidity) significantly influence energy consumption.
- **Random Variables**: Random_variable1 and random_variable2 exhibited low correlation and importance, indicating they are likely not useful for prediction.
- **Temporal Patterns**: Energy consumption varies by hour, day of week, and month, suggesting opportunities for time-based optimization.

## Model Performance
- The Random Forest model demonstrated robust performance with low RMSE and high R², indicating reliable predictions.
- Cross-validation with TimeSeriesSplit confirmed model stability across different temporal splits.

## Recommendations
1. **Optimize Environmental Conditions**: Adjust temperature and humidity in high-impact zones to minimize energy usage, such as improving insulation or HVAC efficiency.
2. **Schedule Operations**: Shift high-energy tasks to off-peak hours or days (e.g., weekends if lower consumption is observed) based on temporal patterns.
3. **Monitor Key Sensors**: Prioritize maintenance and calibration of sensors in critical zones to ensure accurate data for energy management.
4. **Exclude Random Variables**: Discontinue collecting random_variable1 and random_variable2 to streamline data collection, as they add little predictive value.

## Limitations
- The model relies on historical sensor data and assumes consistent data quality; outliers or sensor failures could affect predictions.
- Generalization to other facilities may require retraining with site-specific data.
- The model does not account for sudden operational changes or external factors like equipment upgrades.
    """
    
    with open('insights_report.md', 'w') as f:
        f.write(insights)
    
    print("\nInsights and recommendations saved to 'insights_report.md'")

# Generate insights
generate_insights(df_processed, rf_model, rf_features)

# 7. Save Final Model
# Save the best model (Random Forest) and scaler
joblib.dump(rf_model, 'energy_prediction_model.pkl')
joblib.dump(rf_scaler, 'scaler.pkl')
joblib.dump(selected_features, 'selected_features.pkl')
print("\nModel, scaler, and selected features saved as 'energy_prediction_model.pkl', 'scaler.pkl', and 'selected_features.pkl'")

Dataset Shape: (16857, 29)

Data Types After Cleaning:
timestamp                        object
equipment_energy_consumption    float64
lighting_energy                 float64
zone1_temperature               float64
zone1_humidity                  float64
zone2_temperature               float64
zone2_humidity                  float64
zone3_temperature               float64
zone3_humidity                  float64
zone4_temperature               float64
zone4_humidity                  float64
zone5_temperature               float64
zone5_humidity                  float64
zone6_temperature               float64
zone6_humidity                  float64
zone7_temperature               float64
zone7_humidity                  float64
zone8_temperature               float64
zone8_humidity                  float64
zone9_temperature               float64
zone9_humidity                  float64
outdoor_temperature             float64
atmospheric_pressure            float64
outdoor_humidity         

,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,...,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2,hour,day_of_week,month,is_weekend
count,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,...,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000,16857.000000
mean,93.872992,3.736255,21.253576,39.055580,19.782540,39.535636,21.670575,38.211662,20.242346,37.952298,...,79.223550,4.186361,38.531744,2.781865,24.855569,25.081094,11.506733,2.978881,2.771668,0.276087
std,177.925361,14.228504,2.103842,9.686901,2.862376,9.867143,2.531931,9.876752,2.714872,10.517395,...,27.902210,4.300266,20.799517,5.943530,25.564582,24.892020,6.933696,1.980812,1.179795,0.447074
min,-1139.985693,-86.002966,8.726818,-46.660000,2.988199,-77.265503,6.543921,-71.406273,4.613485,-81.446225,...,-221.668765,-20.929527,-82.329792,-32.098095,-120.170177,-120.409590,0.000000,0.000000,1.000000,0.000000
25%,50.000000,0.000000,20.600000,37.126667,18.666667,37.933333,20.600000,36.754000,19.292500,35.290000,...,72.000000,2.000000,29.000000,0.583333,12.808011,12.805830,5.000000,1.000000,2.000000,0.000000
50%,60.000000,0.000000,21.323333,39.226667,19.600000,40.293333,21.767500,38.400000,20.290000,38.090000,...,84.166667,4.000000,40.000000,2.750000,24.866978,24.834261,12.000000,3.000000,3.000000,0.000000
75%,90.000000,0.000000,22.100000,42.296667,20.700000,42.790000,22.700000,41.000000,21.290000,41.230000,...,91.500000,5.666667,40.000000,5.150000,37.242702,37.277176,18.000000,5.000000,4.000000,1.000000
max,1139.985693,86.002966,33.746609,76.292756,36.552882,77.265503,36.823982,71.406273,35.921144,81.446225,...,221.668765,29.318719,159.606156,37.673716,170.156325,170.329617,23.000000,6.000000,12.000000,1.000000



Selected Features:
['zone1_humidity', 'zone2_temperature', 'zone2_humidity', 'zone3_temperature', 'zone3_humidity', 'zone4_humidity', 'zone5_humidity', 'zone6_temperature', 'zone6_humidity', 'zone7_humidity', 'zone8_temperature', 'zone9_humidity', 'outdoor_temperature', 'atmospheric_pressure', 'dew_point', 'random_variable1', 'hour']

Training Random Forest...

Model Performance (RandomForest):
RMSE: 177.8149
MAE: 95.3271
R²: -0.1854

Cross-Validation R² Scores:
Mean: -0.1407, Std: 0.1074

Training XGBoost...

Model Performance (XGBoost):
RMSE: 192.1318
MAE: 108.4451
R²: -0.3840

Cross-Validation R² Scores:
Mean: -0.3994, Std: 0.1917

Insights and recommendations saved to 'insights_report.md'

Model, scaler, and selected features saved as 'energy_prediction_model.pkl', 'scaler.pkl', and 'selected_features.pkl'
